# Numerical Analysis Project
Team: 아프지마

Member

* 20181628 성창훈
* 20181635 신명철

written by 성창훈, 신명철

# Leaf Rice Disease Dataset

병충해 학습을 위한 이미지는 Kaggle의 Leaf Rice Disease Dataset을 이용한다.

(https://www.kaggle.com/tedisetiady/leaf-rice-disease-indonesia)

총 240장의 .jpg 사진으로 blast, blight, tungro 세 종류의 병충해 사진이 각각 80장씩 있다.

데이터마다 조금씩 차이는 있지만 대략 1440 * 1920 사이즈의 RGB 이미지이다.

## Image to Numpy Ndarray

바이너리 파일인 이미지를 학습에 용이한 형태로 변형시켜야 한다.

이미지의 크기가 너무 큰 경우 학습 시간이 너무 오래걸리거나 학습할 때 지나치게 많은 메모리를 요구한다.

따라서 적당하게 사이즈를 360 * 480 으로 감소시켰다.

병충해 구분에 도움이 될 것 같아서 grayscale을 적용하지 않고 RGB 3채널을 유지했다.

컴퓨터는 0~1사이의 값을 잘 처리한다는 것이 알려져있으므로 RGB값을 255로 나누어 정규화한다.


In [1]:
import numpy as np
from keras.preprocessing.image import img_to_array, load_img
import os

data = []
labels = []
label = -1

for dirname, _, filenames in os.walk('../input/leaf-rice-disease-indonesia'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        img = load_img(path, color_mode='rgb', target_size=(360, 480))
        im = img_to_array(img)/255
        data.append(im)
        labels.append(label)
    label = label + 1

In [2]:
print(data[0])
print("length of data:", len(data))
data = np.array(data)
print("shape of data:", data.shape)


[[[0.8745098  0.91764706 0.99607843]
  [0.8745098  0.91764706 0.99607843]
  [0.8745098  0.91764706 0.99607843]
  ...
  [0.9019608  0.9490196  1.        ]
  [0.9019608  0.9490196  1.        ]
  [0.9019608  0.9490196  1.        ]]

 [[0.8745098  0.91764706 0.99607843]
  [0.8745098  0.91764706 0.99607843]
  [0.8745098  0.91764706 0.99607843]
  ...
  [0.9019608  0.9490196  1.        ]
  [0.9019608  0.9490196  1.        ]
  [0.9019608  0.9490196  1.        ]]

 [[0.8745098  0.91764706 0.99607843]
  [0.8745098  0.91764706 0.99607843]
  [0.8745098  0.91764706 0.99607843]
  ...
  [0.9019608  0.9490196  1.        ]
  [0.9019608  0.9490196  1.        ]
  [0.9019608  0.9490196  1.        ]]

 ...

 [[0.8235294  0.85490197 0.99607843]
  [0.8235294  0.85490197 0.99607843]
  [0.8352941  0.85882354 1.        ]
  ...
  [0.8862745  0.91764706 1.        ]
  [0.8862745  0.91764706 1.        ]
  [0.8784314  0.92156863 1.        ]]

 [[0.8235294  0.85490197 0.99607843]
  [0.8235294  0.85490197 0.99607843]


In [4]:
print("length of labels:", len(labels))
labels = np.array(labels)
print("shape of labels:", labels.shape)

length of labels: 240
shape of labels: (240,)


In [5]:
print(labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


## Export to Numpy .npz file

| ![ex_screenshot](https://miro.medium.com/max/984/1*xwpjjSdZwiOMnPJtdp9L2w.png) |
|:--:|
| *압도적인 퍼포먼스 차이* |

이번 프로젝트에서는 많은 AutoML Framework를 다뤄야 하고, 그럴 때마다 이미지를 따로 전처리하는 것은 비효율적이다.

따라서 Numpy Ndarray로 변환된 이미지 데이터셋을 Numpy 패키지에서 지원하는 형식으로 export한다.

실제로 .txt나 .csv로 변환하는 것에 비해 큰 차이가 있기 때문에 이러한 파일 형식을 적극적으로 활용하기로 했다.

numpy.save()로 단일 배열을 .npy 파일로 저장할 수 있고, numpy.savez()로 여러 배열을 .npz로 저장할 수 있다.

이번 프로젝트에서는 numpy.savez_compressed() 메소드로 압축하여 .npz로 저장했다.

파일을 로드할 때에는 확장자와 관계없이 numpy.load()에 경로를 전달하면 된다.



이미지 출처: https://towardsdatascience.com/what-is-npy-files-and-why-you-should-use-them-603373c78883

In [6]:
np.savez_compressed("imgset(small_rgb).npy",x=data, y=labels)